In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. List all subdirectories in the TCGA root directory
subdirectories = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {subdirectories}")

# The target trait is Hutchinson-Gilford Progeria Syndrome
# Define key terms relevant to Progeria Syndrome
key_terms = ["progeria", "aging", "premature", "gilford", "hutchinson", "skin", "aging", "lamin"]

# Initialize variables for best match
best_match = None
best_match_score = 0
min_threshold = 1  # Require at least 1 matching term

# Convert trait to lowercase for case-insensitive matching
target_trait = trait.lower()  # "hutchinson-gilford_progeria_syndrome"

# Search for relevant directories
for subdir in subdirectories:
    if not os.path.isdir(os.path.join(tcga_root_dir, subdir)) or subdir.startswith('.'):
        continue
        
    subdir_lower = subdir.lower()
    
    # Check for exact matches with key parts of the syndrome name
    if "progeria" in subdir_lower or "hutchinson" in subdir_lower or "gilford" in subdir_lower:
        best_match = subdir
        print(f"Found exact match: {subdir}")
        break
    
    # Calculate score based on key terms
    score = 0
    for term in key_terms:
        if term.lower() in subdir_lower:
            score += 1
    
    # Update best match if score is higher than current best
    if score > best_match_score and score >= min_threshold:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# Handle the case where a match is found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found in TCGA.")

Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc